## 导入必要的库

In [1]:
import torch

## 浮点数加法的非结合性

模拟在多线程环境下，梯度累加顺序不同导致的微小差异

### "大数吃小数" 现象

`(A + small) + small` vs `A + (small + small)`

In [2]:
!nvidia-smi

Sun Nov 30 03:47:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
large_grad = torch.tensor([1e8], dtype=torch.float32).cuda()
small_grad = torch.tensor([1e-5], dtype=torch.float32).cuda()

# 顺序 A: 先加两个小的，再加大数 (模拟线程 1, 2 先完成，3 后完成)
sum_a = (small_grad + small_grad) + large_grad

# 顺序 B: 大数先加小数，再加小数 (模拟线程 3 先完成，1, 2 陆续完成)
sum_b = (large_grad + small_grad) + small_grad

diff = sum_a - sum_b
print(f"Sum A: {sum_a.item():.20f}")
print(f"Sum B: {sum_b.item():.20f}")
print(f"Difference exists: {diff.item() != 0}")

Sum A: 100000000.00000000000000000000
Sum B: 100000000.00000000000000000000
Difference exists: False


In [4]:
# 重新设定数值，利用 float32 在 2^24 (16777216) 处分辨率为 1.0 的特性
large_grad = torch.tensor([16777216.0], dtype=torch.float32).cuda()
small_grad = torch.tensor([0.4], dtype=torch.float32).cuda()

# 顺序 A: 先累加小数，积少成多，足以影响大数
# 0.4 + 0.4 = 0.8
# 16777216.0 + 0.8 -> 16777217.0 (因为 0.8 > 0.5，进位)
sum_a = large_grad + (small_grad + small_grad)

# 顺序 B: 大数直接吃掉小数
# 16777216.0 + 0.4 -> 16777216.0 (因为 0.4 < 0.5，被舍弃)
# 16777216.0 + 0.4 -> 16777216.0 (再次被舍弃)
sum_b = (large_grad + small_grad) + small_grad

diff = sum_a - sum_b
print(f"Sum A (small first): {sum_a.item():.1f}")
print(f"Sum B (large first): {sum_b.item():.1f}")
print(f"Difference: {diff.item()}")

Sum A (small first): 16777216.0
Sum B (large first): 16777216.0
Difference: 0.0


## 理论背景

虽然很难在 Python 简单脚本中稳定复现多线程竞态导致的数值差异（因为现代 CPU/GPU 极快且 Python 有 GIL），但我们可以从数学原理上演示为什么"加法顺序"会导致结果不同，这是 Autograd 多线程非确定性的根源。

### Autograd 引擎的多线程特性

Autograd 引擎在 C++ 层是多线程执行的（例如 Hogwild 模式或并行分支）。多个线程可能几乎同时尝试把梯度加到同一个叶子节点的 `.grad` 上。虽然 AccumulateGrad 有锁（Mutex）保证不会崩溃，但谁先抢到锁是不确定的。

### 浮点数精度限制

代码展示了即使是简单的加法，由于浮点数精度限制（Float32），`(a+b)+c` 并不总是等于 `a+(b+c)`。在深度学习数百万次迭代中，这种微小的"非确定性"是客观存在的。这也是为什么完全复现某些训练结果非常困难的原因之一。